In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet
import pickle

Разделим данные на обучение и тест, нормализуем их

In [2]:
@st.cache
def prepared_dataset(df, random_state=None):
    X = df.drop('price', axis=1)
    y = df.price.astype('int64')
    numeric_cols = np.array(['symboling', 'normalized_losses', 'wheel_base', 'length', 'width',
       'height', 'curb_weight', 'num_of_cylinders', 'engine_size', 'bore',
       'stroke', 'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg',
       'highway_mpg'])
    X_numeric = X[numeric_cols]
    categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))
    X_categorical = X[categorical_cols]
    for col in categorical_cols:
        X_categorical[col] = X_categorical[col].astype('string')
    # Разделим на обучение и тест
    X_train, X_test, X_train_cat, X_test_cat, X_train_num, X_test_num, y_train, y_test = train_test_split(X, X_categorical, X_numeric, y, test_size=0.15, random_state=random_state)
    # Номализуем значения
    scaler = StandardScaler()
    scaler.fit(X_train_num)

    X_train_num_sc = scaler.transform(X_train_num)
    X_test_num_sc = scaler.transform(X_test_num)
    # Соединяем значения воедино
    X_train_transform = np.hstack((X_train_num_sc, X_train_cat))
    X_test_transform = np.hstack((X_test_num_sc, X_test_cat))
    return X_train_transform, y_train, X_test_transform, y_test

# Обучение алгоритмов

In [3]:
def trainee_model(X_train, y_train, parameters={
    'alpha': 1,
    'fit_intercept': True,
    'max_iter': 1,
    'positive': True
}):
    classifier_EN = ElasticNet(alpha=parameters['alpha'], fit_intercept=parameters['fit_intercept'],
                               max_iter=parameters['max_iter'], positive=parameters['positive'])
    classifier_EN.fit(X_train, y_train)
    return classifier_EN

In [4]:
def process_prediction(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    st.write('R^2: {}'.format(r2_score(y_test, y_pred)))
    st.write('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
    st.write('RMSE: {}'.format(mean_squared_error(y_test, y_pred, squared=False)))
    st.write('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))

# Сериализация модели

## Pickle

In [5]:
def serialize_model(model):
    return pickle.dumps(model)

def deserialize_model(bin_data):
    return pickle.loads(bin_data)

Для сериализации/десериализации в/из файла можно использовать следующие функции

In [6]:
def serialize_model_file(model, filename="ser.dat"):
    with open(filename, 'wb') as fin:
        pickle.dump(model, fin)
    return filename

def deserialize_model(filename="ser.dat"):
    with open(filename, 'rb') as fout:
        model = pickle.load(fout)
    return model

# [STREAMLIT](https://www.notion.so/a53e9e35dc4f482f889e2a3f516be9fd)

In [7]:
@st.cache
def load_data(filename='../data/automobile_preprocessed.csv'):
    df = pd.read_csv(filename, sep=';')
    return df

In [8]:
def main():
    page = st.sidebar.selectbox('Choose a page', ['Description', 'Regressor'])
    
    if page == 'Description':
        st.header('Automobile dataset description')
        description_task = '''## Task 3
Подготовьте данные, уберите аномалии. Разбейте датасет, проведите обучение моделей: 
- Линейная регрессия;
- Регрессия дерева решений;
- LASSO;
- Гребневая регрессия;
- Elastic Net регрессия.

Найдите реализации методов в sklearn, оставьте в нотбуке ссылки на документацию. Найдите наилучшие гиперпараметры. Оцените качество моделей: R2, Mean Square Error(MSE), Root Mean Square Error(RMSE),  mean absolute error (MAE). Свои действия снабжайте пояснениями.'''
        st.write(description_task)
        description = '''
        ## Автомобили [Источник](https://archive.ics.uci.edu/ml/datasets/Automobile)

### Описание данных

     Attribute:                Attribute Range:
     ------------------        -----------------------------------------------
  1. symboling:                -3, -2, -1, 0, 1, 2, 3.
  2. normalized-losses:        continuous from 65 to 256.
  3. make:                     alfa-romero, audi, bmw, chevrolet, dodge, honda,
                               isuzu, jaguar, mazda, mercedes-benz, mercury,
                               mitsubishi, nissan, peugot, plymouth, porsche,
                               renault, saab, subaru, toyota, volkswagen, volvo
  4. fuel-type:                diesel, gas.
  5. aspiration:               std, turbo.
  6. num-of-doors:             four, two.
  7. body-style:               hardtop, wagon, sedan, hatchback, convertible.
  8. drive-wheels:             4wd, fwd, rwd.
  9. engine-location:          front, rear.
 10. wheel-base:               continuous from 86.6 120.9.
 11. length:                   continuous from 141.1 to 208.1.
 12. width:                    continuous from 60.3 to 72.3.
 13. height:                   continuous from 47.8 to 59.8.
 14. curb-weight:              continuous from 1488 to 4066.
 15. engine-type:              dohc, dohcv, l, ohc, ohcf, ohcv, rotor.
 16. num-of-cylinders:         eight, five, four, six, three, twelve, two.
 17. engine-size:              continuous from 61 to 326.
 18. fuel-system:              1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.
 19. bore:                     continuous from 2.54 to 3.94.
 20. stroke:                   continuous from 2.07 to 4.17.
 21. compression-ratio:        continuous from 7 to 23.
 22. horsepower:               continuous from 48 to 288.
 23. peak-rpm:                 continuous from 4150 to 6600.
 24. city-mpg:                 continuous from 13 to 49.
 25. highway-mpg:              continuous from 16 to 54.
 26. price:                    continuous from 5118 to 45400.

8. Missing Attribute Values: (denoted by "?")
   Attribute #:   Number of instances missing a value:
   2.             41
   6.             2
   19.            4
   20.            4
   22.            2
   23.            2
   26.            4

### Source Information
   -- Creator/Donor: Jeffrey C. Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
   -- Date: 19 May 1987
   -- Sources:
     1) 1985 Model Import Car and Truck Specifications, 1985 Ward's
        Automotive Yearbook.
     2) Personal Auto Manuals, Insurance Services Office, 160 Water
        Street, New York, NY 10038 
     3) Insurance Collision Report, Insurance Institute for Highway
        Safety, Watergate 600, Washington, DC 20037

### Past Usage
   -- Kibler,~D., Aha,~D.~W., \& Albert,~M. (1989).  Instance-based prediction
      of real-valued attributes.  {\it Computational Intelligence}, {\it 5},
      51--57.
	 -- Predicted price of car using all numeric and Boolean attributes
	 -- Method: an instance-based learning (IBL) algorithm derived from a
	    localized k-nearest neighbor algorithm.  Compared with a
	    linear regression prediction...so all instances
	    with missing attribute values were discarded.  This resulted with
	    a training set of 159 instances, which was also used as a test
	    set (minus the actual instance during testing).
	 -- Results: Percent Average Deviation Error of Prediction from Actual
	    -- 11.84% for the IBL algorithm
	    -- 14.12% for the resulting linear regression equation

4. Relevant Information:
   -- Description
      This data set consists of three types of entities: (a) the
      specification of an auto in terms of various characteristics, (b)
      its assigned insurance risk rating, (c) its normalized losses in use
      as compared to other cars.  The second rating corresponds to the
      degree to which the auto is more risky than its price indicates.
      Cars are initially assigned a risk factor symbol associated with its
      price.   Then, if it is more risky (or less), this symbol is
      adjusted by moving it up (or down) the scale.  Actuarians call this
      process "symboling".  A value of +3 indicates that the auto is
      risky, -3 that it is probably pretty safe.

      The third factor is the relative average loss payment per insured
      vehicle year.  This value is normalized for all autos within a
      particular size classification (two-door small, station wagons,
      sports/speciality, etc...), and represents the average loss per car
      per year.

   -- Note: Several of the attributes in the database could be used as a
            "class" attribute.

        '''
        st.write(description)
    else:
        st.header('Automobile dataset')
        df = load_data('../data/automobile_preprocessed.csv')
        alpha = st.slider('Select param alpha', min_value=0.1, max_value=3.0, value=1.0, step=0.1, key='alpha')
        fit_intercept = st.selectbox('Select fit intercept', [True, False], key='is_fit_intercept')
        max_iter = st.slider('Select param max_iter', min_value=1, max_value=50, value=1, step=1, key='max_iter')
        positive = st.selectbox('Select is all params are positive', [True, False], key='is_positive')
        
        parameters = {
            'alpha': alpha,
            'fit_intercept': fit_intercept,
            'max_iter': max_iter,
            'positive': positive
        }
        X_train, y_train, X_test, y_test = prepared_dataset(df)
        
        model = trainee_model(X_train, y_train, parameters)
        process_prediction(model, X_test, y_test)

In [9]:
if __name__ == "__main__":
    main()

2021-05-18 00:21:48.028 WARNING root: 
  command:

    streamlit run D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
